In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.targeted_attack import Nettack
from scipy.sparse import csr_matrix

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
cora_dataset = Planetoid(root='', name='Pubmed')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [4]:
data_dr = Dataset(root='/tmp/', name='pubmed', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading pubmed dataset...


In [5]:
degrees = np.array(adj.sum(axis=1)).flatten()
max_degree_vertex = degrees.argmax()

In [6]:
adj = csr_matrix(adj)

In [7]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [8]:
ptb_rate = 0.1

In [ ]:
model = Nettack(model=surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(features, adj, labels, max_degree_vertex, n_perturbations=perturbations)
modified_adj = model.modified_adj

##### Starting attack #####
##### Attack only using structure perturbations #####
##### Attacking the node directly #####
##### Performing 4432 perturbations #####
##### ...1/4432 perturbations ... #####


/home/wcorcoran/anaconda3/envs/mesp/lib/python3.8/site-packages/numba/core/ir_utils.py:2172: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected set' found for argument 'edges_set' of function 'compute_new_a_hat_uv'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../anaconda3/envs/mesp/lib/python3.8/site-packages/deeprobust/graph/targeted_attack/nettack.py", line 501:
@jit(nopython=True)
def compute_new_a_hat_uv(edge_ixs, node_nb_ixs, edges_set, twohop_ixs, values_before, degs, potential_edges, u):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


##### ...2/4432 perturbations ... #####
##### ...3/4432 perturbations ... #####
##### ...4/4432 perturbations ... #####
##### ...5/4432 perturbations ... #####
##### ...6/4432 perturbations ... #####
##### ...7/4432 perturbations ... #####
##### ...8/4432 perturbations ... #####
##### ...9/4432 perturbations ... #####
##### ...10/4432 perturbations ... #####
##### ...11/4432 perturbations ... #####
##### ...12/4432 perturbations ... #####
##### ...13/4432 perturbations ... #####
##### ...14/4432 perturbations ... #####
##### ...15/4432 perturbations ... #####
##### ...16/4432 perturbations ... #####
##### ...17/4432 perturbations ... #####
##### ...18/4432 perturbations ... #####
##### ...19/4432 perturbations ... #####
##### ...20/4432 perturbations ... #####
##### ...21/4432 perturbations ... #####
##### ...22/4432 perturbations ... #####
##### ...23/4432 perturbations ... #####
##### ...24/4432 perturbations ... #####
##### ...25/4432 perturbations ... #####
##### ...26/4432 perturb

### arch tests

In [ ]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [ ]:
model_save_path = "../models/pubmed_gcn_model.pth"
list_save_path = "../attacks/cora_gcn_edges.pth" # pubmed doesn't exist yet... have to satiate

In [ ]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [ ]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

In [ ]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [ ]:
arr_diff

In [ ]:
data_modded = copy.deepcopy(data)

In [ ]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

In [ ]:
new_loss, new_accuracy = train.test(data_modded)

print(f"Evasion Change in Accuracy: {(100 * ((new_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Evasion Change in Loss: {(100 * ((new_loss - initial_loss) / initial_loss)):.4f}%")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model_poisoning.reset_parameters()
train_modded = Trainable(model_poisoning)
train_modded.fit(data_modded, 1000)

poisoning_loss, poisoning_accuracy = train_modded.test(data_modded)
print(f"Poisoning Change in Accuracy: {(100 * ((poisoning_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Poisoning Change in Loss: {(100 * ((poisoning_loss - initial_loss) / initial_loss)):.4f}%")